In [26]:
import configparser
import os
import json
from logging.handlers import RotatingFileHandler

import requests
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd
import logging
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy.exc import SQLAlchemyError

In [27]:
url = "https://api.mousephenotype.org/media/J?parameterKey=IMPC_EYE_050_001&genotype=JR28308&start=0&resultsize=50"
response = requests.get(url)
payload = response.json()

In [28]:
payload

{'total': 2,
 'success': True,
 'mediaFiles': [{'url': 'ftp://bhjlk01.jax.org/images/IMPC_EYE_001/929397_929397_left.tif',
   'animalName': 'J49161',
   'genotype': 'JR28308',
   'strain': 'MGI:3056279',
   'parameterKey': 'IMPC_EYE_050_001',
   'procedureKey': 'IMPC_EYE_001',
   'phase': 'tile',
   'status': 'done',
   'xmlFileName': 'J.2022-02-28.63.experiment.impc.xml'},
  {'url': 'ftp://bhjlk01.jax.org/images/IMPC_EYE_001/929397_929397_right.tif',
   'animalName': 'J49161',
   'genotype': 'JR28308',
   'strain': 'MGI:3056279',
   'parameterKey': 'IMPC_EYE_050_001',
   'procedureKey': 'IMPC_EYE_001',
   'phase': 'tile',
   'status': 'done',
   'xmlFileName': 'J.2022-02-28.63.experiment.impc.xml'}]}

In [29]:
dict_ = payload["mediaFiles"][0]

In [30]:
type(dict_)

dict

In [31]:
df = pd.Series(dict_).to_frame()

In [32]:
df

,0
url,ftp://bhjlk01.jax.org/images/IMPC_EYE_001/9293...
animalName,J49161
genotype,JR28308
strain,MGI:3056279
parameterKey,IMPC_EYE_050_001
procedureKey,IMPC_EYE_001
phase,tile
status,done
xmlFileName,J.2022-02-28.63.experiment.impc.xml


In [33]:
res = df.transpose()

In [34]:
res

,url,animalName,genotype,strain,parameterKey,procedureKey,phase,status,xmlFileName
0,ftp://bhjlk01.jax.org/images/IMPC_EYE_001/9293...,J49161,JR28308,MGI:3056279,IMPC_EYE_050_001,IMPC_EYE_001,tile,done,J.2022-02-28.63.experiment.impc.xml


In [35]:
mediaFiles = payload["mediaFiles"]

In [36]:
dataset = []

In [37]:
for mf in mediaFiles:
    temp = pd.Series(mf).to_frame()
    df = temp.transpose()
    dataset.append(df)

In [38]:
res = pd.concat(dataset)

In [39]:
res

,url,animalName,genotype,strain,parameterKey,procedureKey,phase,status,xmlFileName
0,ftp://bhjlk01.jax.org/images/IMPC_EYE_001/9293...,J49161,JR28308,MGI:3056279,IMPC_EYE_050_001,IMPC_EYE_001,tile,done,J.2022-02-28.63.experiment.impc.xml
0,ftp://bhjlk01.jax.org/images/IMPC_EYE_001/9293...,J49161,JR28308,MGI:3056279,IMPC_EYE_050_001,IMPC_EYE_001,tile,done,J.2022-02-28.63.experiment.impc.xml


In [40]:
colNames = list(res.columns)

In [41]:
colNames

['url',
 'animalName',
 'genotype',
 'strain',
 'parameterKey',
 'procedureKey',
 'phase',
 'status',
 'xmlFileName']

In [42]:
User = "chent"
password = "Ql4nc,tzjzsblj"
host = "localhost"
database = "KOMP_Training"

In [43]:
try:
    engine = create_engine("mysql+mysqlconnector://root:{0}@{1}/{2}".
                            format(password, host, database),
                            pool_recycle=1, pool_timeout=57600,
                            future=True)
    res.to_sql("DCCImages", engine, if_exists='append', index=False)

except SQLAlchemyError as e:
    error = str(e.__dict__["orig"])
    print(error)